In [ ]:
import math
import json
from datetime import datetime
import pandas as pd
import numpy as np
import threading

dataFrameTrain = pd.read_csv('../Data/train.csv')
  

def FindIdInColumn(lock,column,callBack,fieldName):
    for i in range(0,len(column)):
        collectionJson = column[i]
        if type(collectionJson) !=str or collectionJson == '':
            continue
        idIndex = 0
        idIndex = collectionJson.find(fieldName,idIndex,len(collectionJson))
        while idIndex != -1:
            idStr = ''
            j = idIndex+5
            while j<len(collectionJson) and collectionJson[j]!=',':
                if not(collectionJson[j].isspace()) and collectionJson[j].isnumeric():
                    idStr = idStr + collectionJson[j]
                j=j+1
            callBack(i,idStr,lock)
            idIndex = idIndex+2
            idIndex = collectionJson.find(fieldName,idIndex,len(collectionJson))

            
def CreateOrAddValueToCol(i,colName,lock):
    lock.acquire()
    dataFrameTrain.at[i,colName] = 1
    lock.release()
        
def AddValueToCollectionColumn(i,value,lock):
    lock.acquire()
    dataFrameTrain.at[i,'belongs_to_collection'] = value
    lock.release()


lock = threading.Lock()
t1 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['genres'],CreateOrAddValueToCol,'\'id\''))
t2 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['production_companies'],CreateOrAddValueToCol,'\'id\''))
t3 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['production_countries'],CreateOrAddValueToCol,'\'name\''))
t4 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['Keywords'],CreateOrAddValueToCol,'\'id\''))
t5 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['Keywords'],CreateOrAddValueToCol,'\'id\''))
t6 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['cast'],CreateOrAddValueToCol,'\'id\''))
t7 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['crew'],CreateOrAddValueToCol,'\'id\''))
t8 = threading.Thread(target=FindIdInColumn,args=(lock,dataFrameTrain['belongs_to_collection'],AddValueToCollectionColumn,'\'id\''))
print('thread start')
t1.start()
t2.start()
t3.start()
t4.start()            
t5.start()
t6.start()
t7.start()
t8.start()                   
t1.join()
t2.join()
t3.join()
t4.join()
print('second batch start')


t5.join()
t6.join()
t7.join()
t8.join()

budgetCollection = dataFrameTrain.budget
for i in range(0,len(budgetCollection)):
    budget = budgetCollection[i]
    if(budget==0):
        dataFrameTrain.loc[i,'budget'] = np.nan
        
dict = {language: id for id, language in enumerate(set(dataFrameTrain.original_language))}
dataFrameTrain.original_language = [dict[language] for language in dataFrameTrain.original_language]

releaseDateCol = dataFrameTrain['release_date']

for i in range(0,len(releaseDateCol)):
    strDate = releaseDateCol[i]
    date = datetime.strptime(strDate, '%m/%d/%y')
    dataFrameTrain.at[i,'day'] = date.day
    dataFrameTrain.at[i,'month'] = date.month
    dataFrameTrain.at[i,'year'] = date.year


thread start
second batch start


In [12]:
import math
import json
from datetime import datetime
import pandas as pd
import numpy as np

dataFrameTrain = pd.read_csv('../Data/train.csv')
def ApplyRow(r):
    print(r.name)
    return dict(r['belongs_to_collection'])['id']
pd.DataFrame(dataFrameTrain['belongs_to_collection'].tolist()).loc[:,'id']

#df = pd.DataFrame(data = dataFrameTrain['belongs_to_collection'])
#df['id'] = df.apply(ApplyRow ,axis =1)


KeyError: 'id'